In [2]:
import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
#from keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

In [3]:
# Prepare the dataset for training and testing
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False, batch_size=100):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [4]:
input_shape = (224,224,3)
image_size = input_shape[0]
# Comment the next line if you want to sweep over different models
# base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')

def train():
    
    config_defaults = {
        'freeze_before' : 50,
        'models' : 'IV3',
        'dropout': 0.2,
        'batch_size': 64,
        'dense_size': 100,
        'num_classes': 10,
        'epochs' : 5,
        'augment_data': True,
        'batch_norm': True,
        'lr': 0.001
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    
    # Defining models and run-name
    if config.models == 'IV3':
        base_model =  keras.applications.InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncV3__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'IRNV2':
        base_model =  keras.applications.InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncResNetV2__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'RN50':
        base_model =  keras.applications.ResNet50(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_ResNet50__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'XCP':
        base_model =  keras.applications.Xception(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_Xception__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)

        
    base_model.trainable = False

    stack_model = Sequential()
    stack_model.add(base_model)
    stack_model.add(Flatten())
    stack_model.add(Dense(config.dense_size, activation='relu'))
    stack_model.add(Dense(config.num_classes, activation='softmax'))
    
    final_model = stack_model
    final_model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    
    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data, batch_size=config.batch_size)
    
    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs=config.epochs,
                    validation_data=val_generator,
                    callbacks=[WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - config.freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False
    
    final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs = 7,
                    validation_data=val_generator,
                    callbacks = [WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

In [5]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        
        'freeze_before' : {
            'values': ['50','70','100']
        },
        'models' :{
            'values' : ['IV3','IRNV2', 'RN50', 'XCP']
        },
        'dropout': {
            'values': [0, 0.1, 0.2, 0.3]
        },     
        'batch_size': {
            'values': [32, 64]
        },
        'dense_size':{
            'values': [64, 100, 128]
        },
        'augment_data': {
            "values": [False, True]
        },
        'batch_norm': {
            "values": [False, True]
        },
        'epochs': {
            "values": [5]
        },
        'lr': {
            "values": [0.005, 0.001, 0.0001]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: 12fb9zkv
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/12fb9zkv


In [6]:
wandb.agent(sweep_id, train, count = 5)

wandb: Agent Starting Run: mt1xkk1c with config:
wandb: 	augment_data: False
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	dense_size: 100
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	freeze_before: 50
wandb: 	lr: 0.001
wandb: 	models: XCP
wandb: Currently logged in as: cs21m029_keyur_raval (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
282/282 [==============================] - 91s 295ms/step - loss: 1.7226 - accuracy: 0.5786 - val_loss: 1.1139 - val_accuracy: 0.6296


C:\Users\Keyur\anaconda3\envs\tf-gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
282/282 [==============================] - 73s 258ms/step - loss: 0.9222 - accuracy: 0.7070 - val_loss: 1.1043 - val_accuracy: 0.6436
Epoch 3/5
282/282 [==============================] - 70s 249ms/step - loss: 0.7325 - accuracy: 0.7577 - val_loss: 1.0950 - val_accuracy: 0.6797
Epoch 4/5
282/282 [==============================] - 70s 249ms/step - loss: 0.5770 - accuracy: 0.8046 - val_loss: 1.0661 - val_accuracy: 0.7127
Epoch 5/5
282/282 [==============================] - 71s 250ms/step - loss: 0.4621 - accuracy: 0.8399 - val_loss: 1.1505 - val_accuracy: 0.7177



accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
val_accuracy,▁▂▅██
val_loss,▅▄▃▁█
accuracy,0.83989
best_epoch,3
best_val_loss,1.06606
epoch,4
loss,0.46207
val_accuracy,0.71772


...

accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▄█▃▆
val_loss,█▄▁█▆
accuracy,0.93511
best_epoch,2
best_val_loss,0.76409
epoch,4
loss,0.19978
val_accuracy,0.76476


Run ik9g9i3v errored: TypeError("unsupported operand type(s) for -: 'int' and 'str'")
wandb: ERROR Run ik9g9i3v errored: TypeError("unsupported operand type(s) for -: 'int' and 'str'")
wandb: Agent Starting Run: 1r9n5vpp with config:
wandb: 	augment_data: True
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	dense_size: 64
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	freeze_before: 50
wandb: 	lr: 0.005
wandb: 	models: IV3


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
282/282 [==============================] - 134s 463ms/step - loss: 4.6148 - accuracy: 0.1491 - val_loss: 2.2098 - val_accuracy: 0.1712
Epoch 2/5
282/282 [==============================] - 131s 464ms/step - loss: 2.1949 - accuracy: 0.1494 - val_loss: 2.1821 - val_accuracy: 0.1702
Epoch 3/5
282/282 [==============================] - 128s 453ms/step - loss: 2.1727 - accuracy: 0.1538 - val_loss: 2.1634 - val_accuracy: 0.1562
Epoch 4/5
282/282 [==============================] - 131s 464ms/step - loss: 2.1527 - accuracy: 0.1653 - val_loss: 2.1406 - val_accuracy: 0.1622
Epoch 5/5
282/282 [==============================] - 128s 452ms/step - loss: 2.1339 - accuracy: 0.1647 - val_loss: 2.1293 - val_accuracy: 0.1522



accuracy,▁▁▃██
epoch,▁▃▅▆█
loss,█▁▁▁▁
val_accuracy,██▂▅▁
val_loss,█▆▄▂▁
accuracy,0.16467
best_epoch,4
best_val_loss,2.1293
epoch,4
loss,2.13394
val_accuracy,0.15215


Run 1r9n5vpp errored: TypeError("unsupported operand type(s) for -: 'int' and 'str'")
wandb: ERROR Run 1r9n5vpp errored: TypeError("unsupported operand type(s) for -: 'int' and 'str'")
